## Full Dataset Pipeline

### Sample inputs dataset

- Samples dataset from hugging face to jsonl file

```uv run scripts/get_dataset_labelled.py --out_path data/refusal/off_policy_raw.jsonl --behaviour refusal --num_samples 1000 --do_label no --do_subsample no```

In [ ]:
from probe_gen.annotation.label_dataset import label_and_save_dataset
from probe_gen.annotation.refusal_behaviour import (
    SYSTEM_PROMPT_REFUSE,
    create_refusal_dataset,
)

dataset = create_refusal_dataset(num_samples=1000)
label_and_save_dataset(
    dataset=dataset,
    dataset_path="../data/refusal/off_policy_raw.jsonl",
    system_prompt=SYSTEM_PROMPT_REFUSE,
    do_subsample=False,
    do_label=False,
)

### Generate outputs dataset (on-policy)

- Uses LLM (Llama-3.2-3B-Instruct default) to generate outputs for inputs dataset
- Takes 5 minutes to do 1000 samples
- Hardware requirements: 40 GB disk, 30-70 GB RAM, 100 GB GPU

```uv run python scripts/get_outputs.py --data data/refusal/off_policy_raw.jsonl --out on_policy_outputs.pkl --batch-size 200 --sample 0 --behaviour refusal```

### Generate outputs dataset (on-policy prompted)

### Label and balance dataset
- Uses GPT-4o API to label refusal behaviour
- Takes 4 minutes to do 10,000 samples
- Hardware requirements: None
- Make sure you have done 'export OPENAI_API_KEY=<key>'

### Get activations dataset

### Upload activations dataset